In [ ]:
import numpy as np
import cv2
from scipy.signal import savgol_filter  # For smoothing
import matplotlib.pyplot as plt

# Assuming you have the 'model' loaded as YOLO('yolov8l-pose.pt')

def compute_speed(avg_positions, fps):
    # Calculate displacement between consecutive frames
    displacement = np.linalg.norm(np.diff(avg_positions, axis=0), axis=1)
    speed = displacement * fps  # speed in units per second
    return speed

def calculate_avg_position(keypoints, side='left'):
    if side == 'left':
        # Average position of left shoulder and left hip
        avg_position = (keypoints[11][:2] + keypoints[5][:2]) / 2
    elif side == 'right':
        # Average position of right shoulder and right hip
        avg_position = (keypoints[12][:2] + keypoints[6][:2]) / 2
    return avg_position

def calculate_angle(a, b, c):
    # a, b, c are 2D points representing the joints (e.g., shoulder, hip, knee)
    ab = a - b
    cb = c - b
    cosine_angle = np.dot(ab, cb) / (np.linalg.norm(ab) * np.linalg.norm(cb))
    angle = np.degrees(np.arccos(cosine_angle))
    return angle

def process_video(file_path, side='left'):
    cap = cv2.VideoCapture(file_path)
    fps = cap.get(cv2.CAP_PROP_FPS)

    avg_positions = []
    thigh_calf_angles = []
    torso_thigh_angles = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Run the YOLO model to get pose keypoints
        results = model(frame)
        largest_person = results[0]['keypoints'][0].cpu().numpy()  # Example to extract keypoints
        
        # Calculate avg position for the side
        avg_position = calculate_avg_position(largest_person, side=side)
        avg_positions.append(avg_position)

        # Calculate angles for left or right side
        if side == 'left':
            # Left side: hip (11), knee (13), ankle (15)
            thigh_calf_angle = calculate_angle(largest_person[11][:2], largest_person[13][:2], largest_person[15][:2])
            torso_thigh_angle = calculate_angle(largest_person[5][:2], largest_person[11][:2], largest_person[13][:2])
        else:
            # Right side: hip (12), knee (14), ankle (16)
            thigh_calf_angle = calculate_angle(largest_person[12][:2], largest_person[14][:2], largest_person[16][:2])
            torso_thigh_angle = calculate_angle(largest_person[6][:2], largest_person[12][:2], largest_person[14][:2])

        thigh_calf_angles.append(thigh_calf_angle)
        torso_thigh_angles.append(torso_thigh_angle)
    
    cap.release()

    avg_positions = np.array(avg_positions)
    return avg_positions, thigh_calf_angles, torso_thigh_angles, fps

# Processing video and getting all data
avg_positions, thigh_calf_angles, torso_thigh_angles, fps = process_video('/mnt/data/squat_segment_1.mp4', side='left')

# Step 1: Calculate speed
speed = compute_speed(avg_positions, fps)

# Step 2: Smoothing the speed for noise reduction
smoothed_speed = savgol_filter(speed, 11, 3)  # Adjust window size and polynomial order as needed

# Step 3: Smoothing the angles for noise reduction
smoothed_thigh_calf = savgol_filter(thigh_calf_angles, 11, 3)
smoothed_torso_thigh = savgol_filter(torso_thigh_angles, 11, 3)

# Step 4: Plotting speed over time
time_axis_speed = np.linspace(0, len(smoothed_speed)/fps, len(smoothed_speed))
plt.figure(figsize=(10, 6))
plt.plot(time_axis_speed, smoothed_speed, label="Speed")
plt.title("Movement Speed During Squat")
plt.xlabel("Time (s)")
plt.ylabel("Speed (units/s)")
plt.legend()
plt.show()

# Step 5: Plotting angles over time
time_axis_angles = np.linspace(0, len(smoothed_thigh_calf)/fps, len(smoothed_thigh_calf))
plt.figure(figsize=(10, 6))
plt.plot(time_axis_angles, smoothed_thigh_calf, label="Thigh-Calf Angle")
plt.plot(time_axis_angles, smoothed_torso_thigh, label="Torso-Thigh Angle")
plt.title("Angle Changes During Squat")
plt.xlabel("Time (s)")
plt.ylabel("Angle (degrees)")
plt.legend()
plt.show()
